# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saynshand,44.8824,110.1163,23.57,37,67,1.75,MN,1722788233
1,1,punta arenas,-53.1500,-70.9167,1.06,86,75,2.57,CL,1722788234
2,2,port lincoln,-34.7333,135.8667,10.64,80,99,0.98,AU,1722788235
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.11,86,62,13.93,SH,1722788236
4,4,pacific grove,36.6177,-121.9166,13.97,97,100,4.92,US,1722788237


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display
# Rename the columns
city_data_df.rename(columns={"Lat": "Latitude", "Lng": "Longitude"}, inplace=True)

# Configure the map plot
map_pot = city_data_df.hvplot.points(
    x= "Longitude",
    y= "Latitude", 
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City")


# Display the map
map_pot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
city_data_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saynshand,44.8824,110.1163,23.57,37,67,1.75,MN,1722788233
1,1,punta arenas,-53.1500,-70.9167,1.06,86,75,2.57,CL,1722788234
2,2,port lincoln,-34.7333,135.8667,10.64,80,99,0.98,AU,1722788235
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.11,86,62,13.93,SH,1722788236
4,4,pacific grove,36.6177,-121.9166,13.97,97,100,4.92,US,1722788237
...,...,...,...,...,...,...,...,...,...,...
561,561,orcopampa,-15.2661,-72.3417,11.85,21,0,2.19,PE,1722788926
562,562,arawa,-6.2132,155.5367,24.23,89,100,1.28,PG,1722788927
563,563,xiaoshi,41.2971,124.1209,24.35,98,100,2.47,CN,1722788928
564,564,londrina,-23.3103,-51.1628,29.81,35,0,4.63,BR,1722788929


In [34]:
# Narrow down cities that fit criteria and drop any results with null values
#Max Temp lower 27 and higher than 21 degrees
city_data_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)]
#wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,fene,43.4500,-8.1500,22.51,65,0,3.81,ES,1722788162
32,32,jamestown,42.0970,-79.2353,26.92,64,0,2.57,US,1722788151
61,61,colonia,50.9333,6.9500,22.75,65,0,2.06,DE,1722788146
90,90,kittila,67.6647,24.8936,22.56,53,0,2.06,FI,1722788342
238,238,tromso,69.6496,18.9570,23.01,46,0,3.09,NO,1722788525
261,261,kolonia,50.9333,6.9500,22.75,65,0,2.06,DE,1722788327
276,276,portland,45.5234,-122.6762,22.65,74,0,0.50,US,1722788450
311,311,kencong,-8.2833,113.3667,22.79,89,0,0.55,ID,1722788616
333,333,glendive,47.1053,-104.7125,24.54,64,0,4.12,US,1722788643
363,363,los corrales de buelna,43.2636,-4.0726,24.58,57,0,2.64,ES,1722788679


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:


# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Latitude', 'Longitude', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display the hotel_df DataFrame


hotel_df

,City,Country,Latitude,Longitude,Humidity,Hotel Name
27,fene,ES,43.4500,-8.1500,65,
32,jamestown,US,42.0970,-79.2353,64,
61,colonia,DE,50.9333,6.9500,65,
90,kittila,FI,67.6647,24.8936,53,
238,tromso,NO,69.6496,18.9570,46,
261,kolonia,DE,50.9333,6.9500,65,
276,portland,US,45.5234,-122.6762,74,
311,kencong,ID,-8.2833,113.3667,89,
333,glendive,US,47.1053,-104.7125,64,
363,los corrales de buelna,ES,43.2636,-4.0726,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fene - nearest hotel: O Allegue
jamestown - nearest hotel: DoubleTree Jamestown
colonia - nearest hotel: Wasserturm Hotel Cologne
kittila - nearest hotel: Hotelli Kittilä
tromso - nearest hotel: Clarion Collection Hotel Aurora
kolonia - nearest hotel: Wasserturm Hotel Cologne
portland - nearest hotel: the Hoxton
kencong - nearest hotel: Grand Gumuk Mas Hotel
glendive - nearest hotel: La Quinta Inn & Suites Glendive
los corrales de buelna - nearest hotel: Hotel Principal del Parque
susanville - nearest hotel: SureStay Plus Hotel by Best Western Susanville
jalapa - nearest hotel: Casa de Juan
hammerfest - nearest hotel: Thon Hotel Hammerfest
jiexiu - nearest hotel: 友谊宾馆
tongchuanshi - nearest hotel: No hotel found


,City,Country,Latitude,Longitude,Humidity,Hotel Name
27,fene,ES,43.4500,-8.1500,65,O Allegue
32,jamestown,US,42.0970,-79.2353,64,DoubleTree Jamestown
61,colonia,DE,50.9333,6.9500,65,Wasserturm Hotel Cologne
90,kittila,FI,67.6647,24.8936,53,Hotelli Kittilä
238,tromso,NO,69.6496,18.9570,46,Clarion Collection Hotel Aurora
261,kolonia,DE,50.9333,6.9500,65,Wasserturm Hotel Cologne
276,portland,US,45.5234,-122.6762,74,the Hoxton
311,kencong,ID,-8.2833,113.3667,89,Grand Gumuk Mas Hotel
333,glendive,US,47.1053,-104.7125,64,La Quinta Inn & Suites Glendive
363,los corrales de buelna,ES,43.2636,-4.0726,57,Hotel Principal del Parque


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display
# Rename the columns
city_data_df.rename(columns={"Lat": "Latitude", "Lng": "Longitude"}, inplace=True)

# Configure the map plot
map_pot_2 = city_data_df.hvplot.points(
    x= "Longitude",
    y= "Latitude", 
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"])


# Display the map
map_pot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country)